In [1]:
import numpy as np
import fractions
import pandas as pd
from fractions import Fraction

# np.set_printoptions(linewidth=200, formatter={'all':lambda x: str(fractions.Fraction(x).limit_denominator())})

In [2]:
def generate_benchmark(alpha: int, beta: int, k: int):
    """Generate benchmark dataset.
    
    alpha determines the number of the column blocks.
    beta  determines the number of the row blocks.
    k     determines the block size
    """
    
    # Construct top right corner k-by-k block
    diag_entries = np.identity(k, dtype=np.double) * ((k-1)/k)
    off_diag_entries = (~np.eye(k,dtype=bool)) * (-1/k)
    block = diag_entries + off_diag_entries
    
    # Construct first layer which corresponds to
    # repeating block entries downwards beta number of times
    first_layer = np.tile(block, (beta, 1))
    
    # Create NxD matrix
    d = alpha * k
    n = beta * k
    data = np.zeros((n, d))
    
    # Fill the last k columns of the matrix
    data[:,-k:] = first_layer
    
    # Fill the rest by using the copy-stack operation
    for j in range(d-k-1, -1, -1):
        for i in range(0, n, 1):
            copy_i = i // k
            copy_j = j + k
            data[i,j] = data[copy_i, copy_j]
    return data

def create_block_highlighter(k: int):
    def apply_highlight(col: pd.Series):
        col_index = int(col.name)
        colors = ['#b3cde0', '#2ab7ca', '#fed766'] if (col_index // k) % 2 == 0 else ['#e6e6ea', '#f6abb6', "#dec3c3"]
        return [f"background-color: {colors[(i // k) %len(colors)]}" for i in range(len(col))]
    return apply_highlight

def highlight_negative(col: pd.Series):
    colors = ['#2ab7ca', '#f6abb6']
    return [f"background-color: {colors[int('-' in val)]}" for val in col.values]


In [3]:
k = 2      # determines the block size
alpha = 3  # determines the number of the column blocks.
beta = 4   # determines the number of the row blocks.

data = generate_benchmark(alpha, beta, k)

df_data = pd.DataFrame(data)
df_data = df_data.apply(lambda r: r.apply(lambda s: str(Fraction(s).limit_denominator())))
df_data.style.apply(create_block_highlighter(k))

,0,1,2,3,4,5
0,1/2,-1/2,1/2,-1/2,1/2,-1/2
1,1/2,-1/2,1/2,-1/2,-1/2,1/2
2,1/2,-1/2,-1/2,1/2,1/2,-1/2
3,1/2,-1/2,-1/2,1/2,-1/2,1/2
4,-1/2,1/2,1/2,-1/2,1/2,-1/2
5,-1/2,1/2,1/2,-1/2,-1/2,1/2
6,-1/2,1/2,-1/2,1/2,1/2,-1/2
7,-1/2,1/2,-1/2,1/2,-1/2,1/2


In [4]:
k = 3      # determines the block size
alpha = 2  # determines the number of the column blocks.
beta = 3   # determines the number of the row blocks.

data = generate_benchmark(alpha, beta, k)

df_data = pd.DataFrame(data)
df_data = df_data.apply(lambda r: r.apply(lambda s: str(Fraction(s).limit_denominator())))
df_data.style.apply(create_block_highlighter(k))

,0,1,2,3,4,5
0,2/3,-1/3,-1/3,2/3,-1/3,-1/3
1,2/3,-1/3,-1/3,-1/3,2/3,-1/3
2,2/3,-1/3,-1/3,-1/3,-1/3,2/3
3,-1/3,2/3,-1/3,2/3,-1/3,-1/3
4,-1/3,2/3,-1/3,-1/3,2/3,-1/3
5,-1/3,2/3,-1/3,-1/3,-1/3,2/3
6,-1/3,-1/3,2/3,2/3,-1/3,-1/3
7,-1/3,-1/3,2/3,-1/3,2/3,-1/3
8,-1/3,-1/3,2/3,-1/3,-1/3,2/3
